```python
def train_input_fn(training_dir, hyperparameters):
    #
def eval_input_fn(training_dir, hyperparameters):
    #
def serving_input_fn(hyperparameters):
    #
def model_fn(features, labels, mode, hyperparameters):
    #
```

In [ ]:
import sagemaker

role    = sagemaker.get_execution_role()
session = sagemaker.Session()
bucket  = session.default_bucket()

In [ ]:
params = {
    'train_tfrecord_file' : 'train.tfr',
    'test_tfrecord_file'  : 'test.tfr',
    'samples_per_epoch'   : 700000,
    'save_summary_steps'  : 100,
}

In [ ]:
your_name = '' # 他の人と被らなければ、なんでも大丈夫です

# 文字列チェック
import re; assert re.match(r'^[0-9a-z-]+$', your_name) is not None

output_path     = 's3://{}/doodle/model/{}/export'.format(bucket, your_name)
checkpoint_path = 's3://{}/doodle/model/{}/ckpt'  .format(bucket, your_name)
code_location   = 's3://{}/doodle/model/{}/source'.format(bucket, your_name)
base_job_name   = 'doodle-training-job-{}'.format(your_name)

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    hyperparameters=params,
    
    output_path=output_path,
    checkpoint_path=checkpoint_path,
    code_location=code_location,
    base_job_name=base_job_name,
    
    source_dir='./src'
    entry_point='doodle.py',
    framework_version='1.6',
    
    role=role,
    training_steps=20000,
    evaluation_steps=2000,
    train_instance_count=1,
    train_instance_type='ml.p2.xlarge')

```python
data_dir = session.upload_data(
    'data', # ローカルディレクトリ
    bucket=bucket,
    key_prefix='doodle/data')
```

In [ ]:
data_dir = 's3://{}/doodle/data'.format(bucket)

In [ ]:
%%time
estimator.fit(data_dir, run_tensorboard_locally=True)

http://(ノートブックのURL)/[proxy/6006/](/proxy/6006/)

In [ ]:
%%time
api = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large')
print(api.endpoint)